## Simple code to test the servos

Inspired by AIY servo examples.

In [1]:
from gpiozero import Servo
from aiy.pins import PIN_A, PIN_B

import time

Create objects for both motors

In [2]:
motor_left = Servo(PIN_A)
motor_right = Servo(PIN_B)

/usr/lib/python3/dist-packages/gpiozero/output_devices.py:1533: PWMSoftwareFallback: To reduce servo jitter, use the pigpio pin factory.See https://gpiozero.readthedocs.io/en/stable/api_output.html#servo for more info
  'To reduce servo jitter, use the pigpio pin factory.'


We need to find a value where the servo does not move (close to zero)

In [7]:
motor_left.value = -0.05
motor_right.value = -0.05

Let's see if we can drive ;)

In [5]:
# Forward
motor_left.value = 0.2
motor_right.value = 0.2

In [6]:
# Stop again
motor_left.value = -0.05
motor_right.value = -0.05